08_Data.csv를 불러와 아래의 미션을 수행해보세요.

1. Tube Furnace CP(소입로 가스 침탄량)과 Corrleation이 높은 상위 5개의 항목을 확인하시오.
2. 앞서 확인한 5개 항목을 X로, Tube Furnace CP을 Y로 선언하여, 전통 통계에서 사용하는 회귀 기법을 이용해 회귀 분석을 실시하시오.
3. 회귀 분석 결과에 대해 아래의 사항을 진단하시오.
    - 아래 진단사항에 가설을 수립하고, 회귀 분석 결과를 이용해 해석하시오.
    - 3-1. 회귀선이 기울기가 존재하고 회귀 선이 특정 경향성을 따르는가?
    - 3-2. 각 회귀 계수들의 값이 회귀식에 대해 유의미한가?
    - 3-3. 회귀 모델의 오차 항이 정규 분포를 따르는가?
4. 회귀식을 이용해, 5개의 값이 각각 순서대로 50, 80, 100, 75, 200 값이 왔을 때, Y값(Tube Furnace CP(소입로 가스 침탄량)을 계산하시오.
5. Z1-OP1(공정건조온도 출력량) 평균에 대해 Z1-OP1 표준편차값이 3배이상 벗어나는 값을 '이상' 나머지를 '정상'으로 'Label'이라는 변수를 생성하고, Code_Num(공정배치그룹)과 'Label' 값이 서로 독립인지 가설 검정을 수행하시오.

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [60]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.stats as stats

In [6]:
df = pd.read_csv("/gdrive/My Drive/Colab Notebooks/하나로/08_Data.csv")
del df['Unnamed: 0']
df

,Code_Num,Datetime,Process_Type,ST,Tube Furnace CP,Tube Furnace1 OP,Tube Furnace1 Temp,Tube Furnace2 OP,Tube Furnace2 Temp,Tube Furnace3 OP,Tube Furnace3 Temp,Tube Furnace4 OP,Tube Furnace4 Temp,Z1-OP1,Z1-OP2,Z1-Temp
0,GroupA,2022-01-03 11:22,OP-A,NaN,0.450497,73.527404,NaN,59.989422,868.759544,52.249481,NaN,72.134908,NaN,74.255730,76.879748,NaN
1,GroupA,2022-01-03 11:22,OP-A,NaN,0.457370,79.149174,878.116929,62.027232,883.821264,50.617109,890.740760,71.245235,894.430606,66.323571,71.297720,NaN
2,GroupA,2022-01-03 11:22,OP-A,68.724636,0.454914,78.946336,866.441979,62.235532,882.850031,52.218350,864.373065,72.705558,893.530501,72.909936,71.579322,99.614201
3,GroupA,2022-01-03 11:22,OP-A,69.161116,0.456540,78.445719,876.350472,61.608260,906.208463,52.863552,891.107865,73.975640,896.347636,68.754355,73.537496,99.376221
4,GroupA,2022-01-03 11:22,OP-C,69.181098,0.460100,76.673808,903.587677,61.589311,867.215131,52.297196,893.206883,71.887865,871.594283,69.692302,72.461153,99.809053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93038,GroupF,2022-01-07 6:17,OP-C,71.033373,0.452082,100.134826,872.637943,46.360373,903.099390,49.074962,907.047411,69.737301,885.226007,79.388710,75.183327,100.349989
93039,GroupF,2022-01-07 6:17,OP-C,70.279919,0.454568,100.495863,859.093360,47.415839,879.563033,48.813327,868.128755,68.664392,907.965662,74.321059,75.029649,100.779761
93040,GroupF,2022-01-07 6:17,OP-C,70.743509,0.449965,100.043239,877.629751,46.868824,878.300413,49.373930,871.801850,69.552292,883.993894,71.790816,76.608828,100.050607
93041,GroupF,2022-01-07 6:17,OP-C,70.135205,0.454477,100.663784,903.544591,46.148800,900.809104,48.564558,880.085060,68.732966,889.400098,65.348224,72.388889,100.295380


In [7]:
#1. Tube Furnace CP(소입로 가스 침탄량)과 Corrleation이 높은 상위 5개의 항목을 확인하시오.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93043 entries, 0 to 93042
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Code_Num            93043 non-null  object 
 1   Datetime            93043 non-null  object 
 2   Process_Type        93043 non-null  object 
 3   ST                  93028 non-null  float64
 4   Tube Furnace CP     93043 non-null  float64
 5   Tube Furnace1 OP    92864 non-null  float64
 6   Tube Furnace1 Temp  93039 non-null  float64
 7   Tube Furnace2 OP    93043 non-null  float64
 8   Tube Furnace2 Temp  93041 non-null  float64
 9   Tube Furnace3 OP    93043 non-null  float64
 10  Tube Furnace3 Temp  93036 non-null  float64
 11  Tube Furnace4 OP    93043 non-null  float64
 12  Tube Furnace4 Temp  93038 non-null  float64
 13  Z1-OP1              93043 non-null  float64
 14  Z1-OP2              93043 non-null  float64
 15  Z1-Temp             93028 non-null  float64
dtypes: f

In [39]:
sample = df[~df.isin(['Code_Num','Datetime','Process_Type'])]
tfcp = abs(sample.corr()).sort_values('Tube Furnace CP',ascending=False).index[1:6]
tfcp

<ipython-input-39-d6254cf578e6>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  tfcp = abs(sample.corr()).sort_values('Tube Furnace CP',ascending=False).index[1:6]


Index(['Tube Furnace3 OP', 'Z1-Temp', 'Tube Furnace1 OP', 'ST',
       'Tube Furnace4 OP'],
      dtype='object')

In [46]:
#2. 앞서 확인한 5개 항목을 X로, Tube Furnace CP을 Y로 선언하여,
# 전통 통계에서 사용하는 회귀 기법을 이용해 회귀 분석을 실시하시오.
dfna = df.dropna()

X = dfna[tfcp]
y = dfna['Tube Furnace CP']

X_const = sm.add_constant(X)
model = sm.OLS(y,X_const)

result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Tube Furnace CP   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     273.1
Date:                Fri, 02 Feb 2024   Prob (F-statistic):          5.48e-291
Time:                        12:44:45   Log-Likelihood:             3.5426e+05
No. Observations:               92847   AIC:                        -7.085e+05
Df Residuals:                   92841   BIC:                        -7.085e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.5305      0.004    131.570      0.000       0.523       0.538
Tube Furnace3 OP     0.0002   9.58e-06     23.968      0.000       0.000       0.000
Z1-Temp             -0.0007   4.01e-05    -16.690      0.000      -0.001      -0.001
Tube Furnace1 OP -1.587e-05   8.43e-07    -18.818      0.000   -1.75e-05   -1.42e-05
ST                  -0.0002   2.91e-05     -6.183      0.000      -0.000      -0.000
Tube Furnace4 OP -8.974e-05   7.48e-06    -11.999      0.000      -0.000   -7.51e-05
==============================================================================
Omnibus:                    43399.570   Durbin-Watson:                   0.597
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1823426.834
Skew:                          -1.553   Prob(JB):                         0.00
Kurtosis:                      24.487   Cond. No.                     3.86e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.86e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

3) 아래 진단사항에 가설을 수립하고, 회귀 분석 결과를 이용해 해석하시오.
1. 회귀선이 기울기가 존재하고 회귀선이 특정 경향성을 따르는가?
- 귀무가설 : 모델이 유의미하지 않다.
- 대립가설 : 모델이 유의미하다.
F통계량의 p값이 0.05보다 작음으로 귀무가설 기각, 이 모델은 전체적으로 유의미하다고 볼 수 있다.


2. 각 회귀 계수들의 값이 회귀식에 대해 유의미한가?
- 귀무가설 : 회귀계수의 값이 0이다
- 대립가설 : 회귀계수의 값이 0이 아니다       
모든 독립 변수들의 p-값이 0.05보다 작음으로 귀무가설 기각, 각 회귀계수들의 값은 유의미하다고 볼 수 있다.

3. 회귀 모델의 오차 항이 정규 분포를 따르는가?
- 귀무가설 : 오차항은 정규분포를 따른다
- 대립가설 : 오차항은 정규분포를 따르지 않는다.     
Omnibus와 JB통계량의 p값이 0.05보다 작음으로, 귀무가설 기각, 오차항이 정규분포를 따르지 않는다고 볼 수 있다.

In [54]:
# 4. 회귀식을 이용해, 5개의 값이 각각 순서대로 50, 80, 100, 75, 200 값이 왔을 때,
# Y값(Tube Furnace CP(소입로 가스 침탄량)을 계산하시오.
0.5305 + 0.0002 * 50 + (-0.0007) * 80 + (-1.587e-05) * 100 + (-0.0002) * 75 + (-8.974e-05) * 200

0.44996499999999995

In [59]:
# 5. Z1-OP1(공정건조온도 출력량) 평균에 대해
# Z1-OP1 표준편차값이 3배이상 벗어나는 값을 '이상' 나머지를 '정상'으로 'Label'이라는 변수를 생성

def outlier(x):
    z1_mean = df['Z1-OP1'].mean()
    z1_std = df['Z1-OP1'].std()

    if x >= z1_mean + 3 * z1_std or x <= z1_mean - 3*z1_std:
        return "이상"
    return '정상'

df['Label'] = df['Z1-OP1'].apply(outlier)
df.head()

,Code_Num,Datetime,Process_Type,ST,Tube Furnace CP,Tube Furnace1 OP,Tube Furnace1 Temp,Tube Furnace2 OP,Tube Furnace2 Temp,Tube Furnace3 OP,Tube Furnace3 Temp,Tube Furnace4 OP,Tube Furnace4 Temp,Z1-OP1,Z1-OP2,Z1-Temp,Label
0,GroupA,2022-01-03 11:22,OP-A,NaN,0.450497,73.527404,NaN,59.989422,868.759544,52.249481,NaN,72.134908,NaN,74.255730,76.879748,NaN,정상
1,GroupA,2022-01-03 11:22,OP-A,NaN,0.457370,79.149174,878.116929,62.027232,883.821264,50.617109,890.740760,71.245235,894.430606,66.323571,71.297720,NaN,정상
2,GroupA,2022-01-03 11:22,OP-A,68.724636,0.454914,78.946336,866.441979,62.235532,882.850031,52.218350,864.373065,72.705558,893.530501,72.909936,71.579322,99.614201,정상
3,GroupA,2022-01-03 11:22,OP-A,69.161116,0.456540,78.445719,876.350472,61.608260,906.208463,52.863552,891.107865,73.975640,896.347636,68.754355,73.537496,99.376221,정상
4,GroupA,2022-01-03 11:22,OP-C,69.181098,0.460100,76.673808,903.587677,61.589311,867.215131,52.297196,893.206883,71.887865,871.594283,69.692302,72.461153,99.809053,정상


In [65]:
# Code_Num(공정배치그룹)과 'Label' 값이 서로 독립인지 가설 검정을 수행하시오.
sample = pd.crosstab(index = df['Code_Num'], columns=df['Label'])
sample

Label,이상,정상
Code_Num,,
GroupA,22,7320
GroupB,30,13269
GroupC,56,26926
GroupD,80,31649
GroupE,10,6208
GroupF,21,7452


In [66]:
stats.chi2_contingency(sample)

Chi2ContingencyResult(statistic=4.74985011160537, pvalue=0.44716509512437286, dof=5, expected_freq=array([[1.72812356e+01, 7.32471876e+03],
       [3.13025268e+01, 1.32676975e+04],
       [6.35088937e+01, 2.69184911e+04],
       [7.46821470e+01, 3.16543179e+04],
       [1.46356201e+01, 6.20336438e+03],
       [1.75895769e+01, 7.45541042e+03]]))

- 귀무가설 : 두 변수는 서로 독립이다
- 대립가설 : 두 변수는 서로 독립이 아니다

카이제곱 p값이 0.447로 귀무가설 채택, 두 변수는 서로 독립이라고 볼 수 있다.